# DoW Raw Data

The purpose of this is to extract day by day information for Frank. Typically just looking at last year. 

Goal: Prove that Friday performs worse than other nights in terms of ticketing.

In [4]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


## Imports

In [5]:
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

%matplotlib inline

In [7]:
os.chdir('/projects/dow/')

## Inputs

## Data Import

Function to import ticketing data

In [11]:
def ticket_data_import(series, fy_start=10, fy_end=19, path='data/'):
    """
    This function retrieves all of the subscription files across a\
 user determined date range and series. 
    
    Keyword arguments:
    series -- either 'clx' or 'pops'
    fy_start -- earliest fiscal year in question (default 10)
    fy_end -- last fiscal year in question (default 19)
    path -- path to data (default 'data/')
    
    Returns:
    df -- pandas dataframe consisting of ticketing data with a fiscal year added to end
    """
    
    accepted_series = ['clx', 'pops']
    
    if series.lower() not in accepted_series:
        raise ValueError('series must be of accepted series types: ', 
                         accepted_series)
    
    dataframes = []
    for i in range(fy_start, fy_end+1):
        file = f"{series.capitalize()}{i}.csv"
        tmp = pd.read_csv(path + file, skiprows=3)
        tmp['fy'] = i
        dataframes.append(tmp)

    df = pd.concat(dataframes, ignore_index=True)
    
    return df

Function to convert dates to datetime

In [17]:
def date_conv(s):
    dates = {date:pd.to_datetime(date) for date in s.unique()}
    return s.map(dates)

Import data

In [14]:
df = ticket_data_import('clx', 17, 19)

Convert perf date to datetime date

In [21]:
df['perf_dt'] = date_conv(df['perf_dt'])

Add a column for day of week

In [33]:
df['dow'] = df['perf_dt'].apply(lambda date: date.strftime('%A'))

In [34]:
df.head()

,perf_code,perf_no,perf_dt,zone_no,zone_desc,section,row,seat,home_price,paid_amt,...,price_type_group_id,price_type_group,pd_up,season,season_desc,summary_cust_id,summary_cust_name,attended,fy,dow
0,PS092216,13971,2016-09-22 20:00:00,4172,Price 2 Orchestra Terr Left,ORCH T,Z,5,85.0,NaN,...,NaN,NaN,AVL,487,PS 16-17 Classics,NaN,NaN,NaN,17,Thursday
1,PS092216,13971,2016-09-22 20:00:00,4194,Price 5 Orchestra Terr Right,ORCH T,X,6,25.0,NaN,...,NaN,NaN,AVL,487,PS 16-17 Classics,NaN,NaN,NaN,17,Thursday
2,PS092216,13971,2016-09-22 20:00:00,4158,Price 1 Orchestra,ORCH,K,104,115.0,NaN,...,NaN,NaN,AVL,487,PS 16-17 Classics,NaN,NaN,NaN,17,Thursday
3,PS092216,13971,2016-09-22 20:00:00,4169,Price 4 Accessible Orchestra,ORCH,B,103,40.0,NaN,...,NaN,NaN,AVL,487,PS 16-17 Classics,NaN,NaN,NaN,17,Thursday
4,PS092216,13971,2016-09-22 20:00:00,4197,Price 4 Promenade Circle,PROM,D,429,40.0,NaN,...,NaN,NaN,AVL,487,PS 16-17 Classics,NaN,NaN,NaN,17,Thursday


## Group by Date

In [40]:
dow_df = df.groupby(['fy', 'dow', 'perf_dt']).agg({
    'summary_cust_id': ['count'],
    'paid_amt': ['sum'],
    'attended': ['count']
}).reset_index()

## Write Data to Excel

In [42]:
with pd.ExcelWriter('dow_raw.xlsx') as writer:
    pd.DataFrame(dow_df).to_excel(writer,
                                  engine='xlsxwriter', 
                                  index=True)